# Geolocation of rental properties

This notebook is used to visualize the **geolocation** of the **rental properties** on an SA2 Shapefile.

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster

In [2]:
# Load the dataset
zones = gpd.read_file( "../data/raw/external/SA2_2021_ShapeFile/SA2_2021_AUST_GDA2020.shp")
rent_df = pd.read_csv("../data/curated/rent_cleaned.csv")

zones = zones.loc[zones['GCC_NAME21'].isin(['Rest of Vic.','Greater Melbourne'])]
zones['geometry'] = zones['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [3]:
zones.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
644,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."
645,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.81896 -37.55582, 143.81644 -37.5..."
646,201011005,Buninyong,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,51.5855,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6..."
647,201011006,Delacombe,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,34.1607,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.7505 -37.59119, 143.75044 -37.59..."
648,201011007,Smythes Creek,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,104.7274,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.73296 -37.62333, 143.73263 -37.6..."


In [4]:
# Create geometry column from latitude and longitude
geometry = [Point(xy) for xy in zip(rent_df['longitude'], rent_df['latitude'])]

# Convert to GeoDataFrame
rent_gdf = gpd.GeoDataFrame(rent_df, geometry=geometry)
rent_gdf.head()

,Unnamed: 0,suburb,postcode,latitude,longitude,isNewDevelopment,bathrooms,bedrooms,carspaces,propertyTypes,status,channel,price_per_week,geometry
0,0,Mount Martha,3934,-38.268818,145.014034,False,3,3,2,house,live,residential,1600.0,POINT (145.01403 -38.26882)
1,1,Bentleigh East,3165,-37.909091,145.055152,False,2,3,2,house,live,residential,780.0,POINT (145.05515 -37.90909)
2,2,Ferntree Gully,3156,-37.890452,145.267922,False,1,3,2,house,live,residential,585.0,POINT (145.26792 -37.89045)
3,3,Frankston North,3200,-38.115353,145.163260,False,1,3,2,house,live,residential,475.0,POINT (145.16326 -38.11535)
4,4,Camberwell,3124,-37.840108,145.094830,False,1,2,1,house,live,residential,590.0,POINT (145.09483 -37.84011)


In [5]:
# Check if each property is within any zone
rent_gdf['zone'] = None  # Add a new column to store the zone name or ID

for index, zone in zones.iterrows():
    mask = rent_gdf.within(zone['geometry'])  # Check if property is within the zone
    rent_gdf.loc[mask, 'zone'] = zone['SA2_CODE21']  

rent_gdf['latitude'] = rent_gdf.geometry.y
rent_gdf['longitude'] = rent_gdf.geometry.x

rent_gdf.head()

,Unnamed: 0,suburb,postcode,latitude,longitude,isNewDevelopment,bathrooms,bedrooms,carspaces,propertyTypes,status,channel,price_per_week,geometry,zone
0,0,Mount Martha,3934,-38.268818,145.014034,False,3,3,2,house,live,residential,1600.0,POINT (145.01403 -38.26882),214021382
1,1,Bentleigh East,3165,-37.909091,145.055152,False,2,3,2,house,live,residential,780.0,POINT (145.05515 -37.90909),208021426
2,2,Ferntree Gully,3156,-37.890452,145.267922,False,1,3,2,house,live,residential,585.0,POINT (145.26792 -37.89045),211011448
3,3,Frankston North,3200,-38.115353,145.163260,False,1,3,2,house,live,residential,475.0,POINT (145.16326 -38.11535),214011372
4,4,Camberwell,3124,-37.840108,145.094830,False,1,2,1,house,live,residential,590.0,POINT (145.09483 -37.84011),207011149


In [6]:
# Create a base map
m = folium.Map(location=[rent_gdf['latitude'].mean(), rent_gdf['longitude'].mean()], zoom_start=10)

# Add zones as GeoJson
folium.GeoJson(zones, name='Zones', style_function=lambda x: {'fillColor': 'lightblue', 'color': 'black', 'weight': 2}).add_to(m)

# Add rental properties with MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
for idx, row in rent_gdf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Zone: {row['zone']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
    ).add_to(marker_cluster)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map
m.save("../plots/rental_properties_map.html")  